In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from dataclasses import dataclass
from typing import List, Tuple, Dict
import json
import shutil
import matplotlib.gridspec as gridspec
import io
from PIL import Image
from IPython.display import Image as IPImage
import torch
from PIL import Image as PILImage
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML
import ipywidgets as widgets
from IPython.display import display, IFrame
from matplotlib.animation import HTMLWriter
from matplotlib.patches import Circle

In [2]:
class Entity:
    def __init__(self, x, y, entity_type, direction=None):
        self.x = x
        self.y = y
        self.type = entity_type
        # direction is a tuple (dx, dy) representing movement direction
        self.direction = direction if direction is not None else (0, 0)

    def move(self):
        self.x += self.direction[0]
        self.y += self.direction[1]
        
    def reverse_direction(self):
        """Reverse the direction by changing signs of dx and dy"""
        self.direction = (-self.direction[0], -self.direction[1])

def detect_collision(entities):
    collisions = []
    for i in range(len(entities)):
        for j in range(i+1, len(entities)):
            if entities[i].x == entities[j].x and entities[i].y == entities[j].y:
                collisions.append((entities[i], entities[j]))
    return collisions


In [3]:
class SyntheticDataGenerator:
    def __init__(self, grid_size=10, num_vehicles=5, num_pedestrians=5, num_obstacles=5):
        self.grid_size = grid_size
        self.num_vehicles = num_vehicles
        self.num_pedestrians = num_pedestrians
        self.num_obstacles = num_obstacles

    def get_horizontal_direction(self):
        """Returns a random horizontal direction"""
        return (np.random.choice([-1, 1]), 0)

    def get_vertical_direction(self):
        """Returns a random vertical direction"""
        return (0, np.random.choice([-1, 1]))

    def will_hit_edge(self, x, y):
        """Check if next position would be at or beyond grid edge"""
        return (x <= 0 or x >= self.grid_size - 1 or 
                y <= 0 or y >= self.grid_size - 1)

    def get_safe_obstacle_position(self, pedestrians, occupied_positions):
        """
        Get a safe position for an obstacle that won't interfere with pedestrian paths
        pedestrians: list of tuples (position, direction)
        occupied_positions: set of (x, y) positions already taken
        """
        max_attempts = 100000
        attempts = 0
        
        while attempts < max_attempts:
            x = np.random.randint(0, self.grid_size)
            y = np.random.randint(0, self.grid_size)
            position = (x, y)
            
            if position in occupied_positions:
                attempts += 1
                continue
                
            # Check if position is safe for all pedestrians
            is_safe = True
            for ped_pos, ped_dir in pedestrians:
                # If pedestrian moves vertically (dx=0), obstacle can't have same x-coordinate
                if ped_dir[0] == 0:  # Vertical movement
                    if x == ped_pos[0]:
                        is_safe = False
                        break
                # If pedestrian moves horizontally (dy=0), obstacle can't have same y-coordinate
                elif ped_dir[1] == 0:  # Horizontal movement
                    if y == ped_pos[1]:
                        is_safe = False
                        break
            
            if is_safe:
                return (x, y)
            
            attempts += 1
            
        # If we couldn't find a safe position, try one last time without checking pedestrian paths
        while True:
            x = np.random.randint(0, self.grid_size)
            y = np.random.randint(0, self.grid_size)
            position = (x, y)
            if position not in occupied_positions:
                return position

    def generate_data(self, num_samples=100, num_steps=10):
        data = []
        for _ in range(num_samples):
            scenario = []
            entities = []
            occupied_positions = set()
            
            # First, place and set up pedestrians
            pedestrian_info = []  # Store pedestrian positions and directions
            vehicles_horizontal = np.random.choice([True, False])
            
            for _ in range(self.num_pedestrians):
                while True:
                    x, y = np.random.randint(0, self.grid_size, size=2)
                    if (x, y) not in occupied_positions:
                        break
                
                # Direction perpendicular to vehicles
                direction = self.get_horizontal_direction() if not vehicles_horizontal else self.get_vertical_direction()
                pedestrian_info.append(((x, y), direction))
                occupied_positions.add((x, y))
                entities.append(Entity(x, y, 'pedestrian', direction))
            
            # Then place vehicles
            for _ in range(self.num_vehicles):
                while True:
                    x, y = np.random.randint(0, self.grid_size, size=2)
                    if (x, y) not in occupied_positions:
                        break
                
                direction = self.get_horizontal_direction() if vehicles_horizontal else self.get_vertical_direction()
                occupied_positions.add((x, y))
                entities.append(Entity(x, y, 'vehicle', direction))
            
            # Finally, place obstacles in safe positions
            for _ in range(self.num_obstacles):
                x, y = self.get_safe_obstacle_position(pedestrian_info, occupied_positions)
                occupied_positions.add((x, y))
                entities.append(Entity(x, y, 'obstacle', (0, 0)))
            
            scenario.append(entities.copy())
            
            # Generate movements for subsequent steps
            for _ in range(num_steps - 1):
                new_entities = []
                for entity in entities:
                    # Create new entity instance with same properties
                    new_entity = Entity(entity.x, entity.y, entity.type, entity.direction)
                    
                    if new_entity.type != 'obstacle':
                        # Check if next position would hit edge
                        next_x = new_entity.x + new_entity.direction[0]
                        next_y = new_entity.y + new_entity.direction[1]
                        
                        # If will hit edge, reverse direction
                        if self.will_hit_edge(next_x, next_y):
                            new_entity.reverse_direction()
                        
                        # Move in current direction (which might have just been reversed)
                        new_entity.move()
                    
                    new_entities.append(new_entity)
                
                scenario.append(new_entities)
                entities = new_entities
                
            data.append(scenario)
        return data

    def visualize_data(self, scenario, sample_id):
        # Create figure with adjusted size to accommodate legend
        fig = plt.figure(figsize=(8, 6))
        
        # Create gridspec to position the plot and legend
        gs = gridspec.GridSpec(1, 2, width_ratios=[4, 1])
        
        # Create main plot and legend axes
        ax = plt.subplot(gs[0])
        legend_ax = plt.subplot(gs[1])
        legend_ax.axis('off')  # Hide the legend axes frame
        
        # Dictionary to store frame-specific collision locations
        collision_data = {}
        
        # Pre-compute collision frames and locations
        for frame, entities in enumerate(scenario):
            collisions = detect_collision(entities)
            if collisions:
                # Store only the exact collision locations for this frame
                collision_data[frame] = [(entity1.x, entity1.y) for entity1, entity2 in collisions]

        def update(frame):
            ax.clear()
            legend_ax.clear()
            legend_ax.axis('off')

            entities = scenario[frame]

            # Create empty lists for legend handles and labels
            legend_elements = []

            # Draw entities and collect unique elements for legend
            vehicles = ax.scatter([], [], color='blue', marker='^', s=200, label='Vehicle')
            pedestrians = ax.scatter([], [], color='red', marker='o', s=200, label='Pedestrian')
            obstacles = ax.scatter([], [], color='green', marker='s', s=200, label='Obstacle')

            # Add to legend elements
            legend_elements.extend([vehicles, pedestrians, obstacles])

            # Draw actual entities and their direction indicators
            for entity in entities:
                if entity.type == 'vehicle':
                    ax.scatter(entity.x, entity.y, color='blue', marker='^', s=200)
                    # Add direction indicator
                    if entity.direction != (0, 0):
                        ax.arrow(entity.x, entity.y, 
                               entity.direction[0] * 0.3, entity.direction[1] * 0.3,
                               head_width=0.1, head_length=0.1, fc='blue', ec='blue')
                elif entity.type == 'pedestrian':
                    ax.scatter(entity.x, entity.y, color='red', marker='o', s=200)
                    # Add direction indicator
                    if entity.direction != (0, 0):
                        ax.arrow(entity.x, entity.y, 
                               entity.direction[0] * 0.3, entity.direction[1] * 0.3,
                               head_width=0.1, head_length=0.1, fc='red', ec='red')
                else:
                    ax.scatter(entity.x, entity.y, color='green', marker='s', s=200)

            # Draw collision circles only at exact collision locations
            if frame in collision_data:
                for x, y in collision_data[frame]:
                    circle = Circle((x, y), radius=1, color='red', alpha=0.3)
                    ax.add_patch(circle)
                ax.text(0.02, 0.98, 'COLLISION!', transform=ax.transAxes, 
                       color='red', fontsize=12, verticalalignment='top')
                # Add collision indicator to legend
                collision_patch = plt.scatter([], [], c='red', alpha=0.3, marker='o', s=500, label='Collision Area')
                legend_elements.append(collision_patch)

            # Set up the main plot with explicit grid lines and labels
            ax.set_xlim(-0.5, self.grid_size - 0.5)
            ax.set_ylim(-0.5, self.grid_size - 0.5)
            ax.set_title(f"Time Step {frame+1}")

            # Set major ticks at integer positions
            ax.set_xticks(range(self.grid_size))
            ax.set_yticks(range(self.grid_size))

            # Add grid with integer spacing
            ax.grid(True, which='major', linestyle='-', linewidth=1)

            # Add minor gridlines at 0.5 intervals if desired
            ax.grid(True, which='minor', linestyle=':', linewidth=0.5)

            # Set labels for axes
            ax.set_xlabel('X Coordinate')
            ax.set_ylabel('Y Coordinate')

            # Create legend in the separate axis
            legend_ax.legend(handles=legend_elements, 
                           labels=['Vehicle', 'Pedestrian', 'Obstacle', 'Collision Area'] if frame in collision_data 
                           else ['Vehicle', 'Pedestrian', 'Obstacle'],
                           loc='center left',
                           bbox_to_anchor=(0, 0.5))

            # Adjust layout to prevent overlap
            plt.tight_layout()

        ani = FuncAnimation(fig, update, frames=len(scenario), interval=500, repeat=False)

        # Create the animations folder if it doesn't exist
        os.makedirs('animations', exist_ok=True)

        # Save the animation as an HTML file
        html_path = f'animations/scenario_{sample_id}_perpendicular.html'
        writer = HTMLWriter(fps=2)
        ani.save(html_path, writer=writer)

        plt.close(fig)
        return html_path

    def analyze_collisions(self, scenario):
        collision_report = []
        for timestep, entities in enumerate(scenario):
            collisions = detect_collision(entities)
            if collisions:
                for entity1, entity2 in collisions:
                    collision_report.append({
                        'timestep': timestep + 1,
                        'location': (entity1.x, entity1.y),
                        'entities': (entity1.type, entity2.type)
                    })
        return collision_report

In [4]:
# Initialize the base generator
base_generator = SyntheticDataGenerator(grid_size=10, num_vehicles=1, num_pedestrians=1, num_obstacles=0)

In [5]:
data = base_generator.generate_data(num_samples=1, num_steps=100)

for i, scenario in enumerate(data):
    print(f"\nScenario {i+1}:")
    html_path = base_generator.visualize_data(scenario, sample_id=i+1)
    display(IFrame(src=html_path, width=800, height=800))

    # Get and display detailed collision report
    collision_report = base_generator.analyze_collisions(scenario)
    if collision_report:
        print("\nCollision Report:")
        for collision in collision_report:
            print(f"Time Step {collision['timestep']}: "
                  f"{collision['entities'][0].capitalize()} collided with "
                  f"{collision['entities'][1]} at position {collision['location']}")
    else:
        print("No collisions detected in this scenario")
    print("-" * 50)


Scenario 1:


No collisions detected in this scenario
--------------------------------------------------


# Dataset Generation begins here

In [6]:

def convert_to_serializable(obj):
    """Convert numpy types to Python native types for JSON serialization"""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, tuple):
        return tuple(convert_to_serializable(item) for item in obj)
    elif isinstance(obj, list):
        return [convert_to_serializable(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: convert_to_serializable(value) for key, value in obj.items()}
    return obj

@dataclass
class Sample:
    """Class to store information about a sequence of frames"""
    frames: List[List[Entity]]  # consecutive frames
    has_collision: bool
    collision_details: Dict = None  # Only for collision samples
    sample_id: int = None
    
    def set_id(self, new_id: int):
        """Set the sample ID"""
        self.sample_id = new_id
        return self

In [7]:
class GNNDatasetGenerator:
    def __init__(self, base_generator: SyntheticDataGenerator):
        self.generator = base_generator
        self.dataset_root = "traffic_dataset"
        self.collision_dir = os.path.join(self.dataset_root, "collision")
        self.no_collision_dir = os.path.join(self.dataset_root, "no_collision")
        self.metadata_path = os.path.join(self.dataset_root, "metadata.json")
        
    def setup_directories(self):
        """Create necessary directories and clean existing data"""
        if os.path.exists(self.dataset_root):
            shutil.rmtree(self.dataset_root)
        
        os.makedirs(self.dataset_root)
        os.makedirs(self.collision_dir)
        os.makedirs(self.no_collision_dir)

    def save_frame(self, entities: List[Entity], path: str, collision_location=None) -> None:
        """Save a single frame as an image with direction indicators and collision marking"""
        fig, ax = plt.subplots(figsize=(8, 8))
        
        # Plot each entity with direction indicators
        for entity in entities:
            if entity.type == 'vehicle':
                ax.scatter(entity.x, entity.y, color='blue', marker='^', s=200)
                if entity.direction != (0, 0):
                    ax.arrow(entity.x, entity.y, 
                           entity.direction[0] * 0.3, entity.direction[1] * 0.3,
                           head_width=0.1, head_length=0.1, fc='blue', ec='blue')
            elif entity.type == 'pedestrian':
                ax.scatter(entity.x, entity.y, color='red', marker='o', s=200)
                if entity.direction != (0, 0):
                    ax.arrow(entity.x, entity.y, 
                           entity.direction[0] * 0.3, entity.direction[1] * 0.3,
                           head_width=0.1, head_length=0.1, fc='red', ec='red')
            else:  # obstacle
                ax.scatter(entity.x, entity.y, color='green', marker='s', s=200)

        # Add collision circle if collision location is provided
        if collision_location:
            x, y = collision_location
            circle = Circle((x, y), radius=1, color='red', alpha=0.3)
            ax.add_patch(circle)
            ax.text(0.02, 0.98, 'COLLISION!', transform=ax.transAxes, 
                   color='red', fontsize=12, verticalalignment='top')
        
        # Set up the plot
        ax.set_xlim(-0.5, self.generator.grid_size - 0.5)
        ax.set_ylim(-0.5, self.generator.grid_size - 0.5)
        ax.grid(True)
        ax.set_xticks(range(self.generator.grid_size))
        ax.set_yticks(range(self.generator.grid_size))
        
        # Save and close
        plt.savefig(path)
        plt.close(fig)

    def check_sequence_validity(self, frames: List[List[Entity]], check_all: bool = False) -> bool:
        """
        Check if a sequence of frames is valid:
        - For collision samples (check_all=False): only last frame should have collision
        - For no-collision samples (check_all=True): no frame should have collision
        """
        for i, frame in enumerate(frames):
            has_collision = len(detect_collision(frame)) > 0
            if check_all and has_collision:
                return False
            elif not check_all and i < len(frames)-1 and has_collision:
                return False
            elif not check_all and i == len(frames)-1 and not has_collision:
                return False
        return True

    def extract_collision_samples(self, scenario: List[List[Entity]], window_size: int = 7) -> List[Sample]:
        """Extract valid collision samples from a scenario"""
        samples = []
        
        for i in range(window_size-1, len(scenario)):
            sequence = scenario[i-window_size+1:i+1]
            if self.check_sequence_validity(sequence, check_all=False):
                collisions = detect_collision(sequence[-1])
                collision_info = {
                    "frame": convert_to_serializable(i),
                    "collisions": [
                        {
                            "types": (c[0].type, c[1].type),
                            "location": (convert_to_serializable(c[0].x), 
                                       convert_to_serializable(c[0].y))
                        } for c in collisions
                    ]
                }
                samples.append(Sample(sequence, True, collision_info))
        
        return samples

    def extract_no_collision_samples(self, scenario: List[List[Entity]], window_size: int = 7) -> List[Sample]:
        """Extract valid no-collision samples from a scenario"""
        samples = []
        
        for i in range(window_size-1, len(scenario)):
            sequence = scenario[i-window_size+1:i+1]
            if self.check_sequence_validity(sequence, check_all=True):
                samples.append(Sample(sequence, False, None))
        
        return samples

    



    def get_scenario_hash(self, first_frame):
        """Create a hash of initial positions and directions to check for uniqueness"""
        positions = []
        for entity in first_frame:
            positions.append((entity.x, entity.y, entity.direction[0], entity.direction[1], entity.type))
        return tuple(sorted(positions))

    def generate_dataset(self, num_samples: int = 100, num_steps: int = 500, window_size: int = 7):
        """Generate complete dataset with balanced collision and no-collision samples"""
        self.setup_directories()
        
        collision_samples = []
        no_collision_samples = []
        samples_per_category = num_samples // 2
        #max_attempts = samples_per_category * 20  # Increased max attempts
        max_attempts = 100000
        attempts = 0
        
        # Keep track of used initial configurations
        used_configurations = set()
        
        while (len(collision_samples) < samples_per_category or 
               len(no_collision_samples) < samples_per_category) and attempts < max_attempts:
            attempts += 1
            
            # Randomize the seed for each generation
            np.random.seed(None)  # Reset the seed
            seed = np.random.randint(0, 1000000)
            np.random.seed(seed)
            
            # Create a new generator instance with randomized parameters
            current_generator = SyntheticDataGenerator(
                grid_size=self.generator.grid_size,
                num_vehicles=self.generator.num_vehicles,
                num_pedestrians=self.generator.num_pedestrians,
                num_obstacles=self.generator.num_obstacles
            )
            
            # Generate a single scenario
            scenario = current_generator.generate_data(num_samples=1, num_steps=num_steps)[0]
            
            # Check for collision samples if needed
            if len(collision_samples) < samples_per_category:
                new_samples = self.extract_collision_samples(scenario, window_size)
                for sample in new_samples:
                    if len(collision_samples) < samples_per_category:
                        # Check if this configuration was used before
                        config_hash = self.get_scenario_hash(sample.frames[0])
                        if config_hash not in used_configurations:
                            sample.set_id(len(collision_samples))
                            collision_samples.append(sample)
                            used_configurations.add(config_hash)
                            print(f"Found new collision sample: {len(collision_samples)}/{samples_per_category}")
            
            # Check for no-collision samples if needed
            if len(no_collision_samples) < samples_per_category:
                new_samples = self.extract_no_collision_samples(scenario, window_size)
                for sample in new_samples:
                    if len(no_collision_samples) < samples_per_category:
                        # Check if this configuration was used before
                        config_hash = self.get_scenario_hash(sample.frames[0])
                        if config_hash not in used_configurations:
                            sample.set_id(len(no_collision_samples))
                            no_collision_samples.append(sample)
                            used_configurations.add(config_hash)
                            print(f"Found new no-collision sample: {len(no_collision_samples)}/{samples_per_category}")
            
            # Print progress every 100 attempts
            if attempts % 100 == 0:
                print(f"Attempt {attempts}: Found {len(collision_samples)} collision and "
                      f"{len(no_collision_samples)} no-collision samples")
        
        if attempts >= max_attempts:
            print(f"Warning: Reached maximum attempts ({max_attempts}). "
                  f"Generated {len(collision_samples)} collision samples and "
                  f"{len(no_collision_samples)} no-collision samples.")
        
        # Save all samples
        print(f"Saving {len(collision_samples)} collision samples...")
        for sample in collision_samples:
            self.save_sample(sample)
            
        print(f"Saving {len(no_collision_samples)} no-collision samples...")
        for sample in no_collision_samples:
            self.save_sample(sample)
        
        # Save global metadata
        global_metadata = {
            "total_samples": convert_to_serializable(len(collision_samples) + len(no_collision_samples)),
            "collision_samples": convert_to_serializable(len(collision_samples)),
            "no_collision_samples": convert_to_serializable(len(no_collision_samples)),
            "window_size": convert_to_serializable(window_size),
            "grid_size": convert_to_serializable(self.generator.grid_size),
            "num_vehicles": convert_to_serializable(self.generator.num_vehicles),
            "num_pedestrians": convert_to_serializable(self.generator.num_pedestrians),
            "num_obstacles": convert_to_serializable(self.generator.num_obstacles),
            "unique_configurations": convert_to_serializable(len(used_configurations))
        }
        
        with open(self.metadata_path, 'w') as f:
            json.dump(global_metadata, f, indent=2)
            
        print("Dataset generation completed!")

    def save_sample(self, sample: Sample):
        """Save a sample's frames and metadata"""
        if sample.sample_id is None:
            raise ValueError("Sample ID must be set before saving")
            
        # Determine target directory
        base_dir = self.collision_dir if sample.has_collision else self.no_collision_dir
        sample_dir = os.path.join(base_dir, f"sample_{sample.sample_id:05d}")
        os.makedirs(sample_dir)
        
        # Save frames
        for i, frame in enumerate(sample.frames):
            frame_path = os.path.join(sample_dir, f"frame_{i:02d}.png")
            
            # Get collision location for the last frame if it's a collision sample
            collision_location = None
            if sample.has_collision and i == len(sample.frames) - 1 and sample.collision_details:
                collision_location = sample.collision_details["collisions"][0]["location"]
                
            self.save_frame(frame, frame_path, collision_location)
        
        # Save initial configuration details
        initial_config = {
            "sample_id": convert_to_serializable(sample.sample_id),
            "has_collision": sample.has_collision,
            "collision_details": convert_to_serializable(sample.collision_details),
            "initial_positions": [
                {
                    "type": entity.type,
                    "position": (convert_to_serializable(entity.x), convert_to_serializable(entity.y)),
                    "direction": (convert_to_serializable(entity.direction[0]), 
                                convert_to_serializable(entity.direction[1]))
                }
                for entity in sample.frames[0]
            ]
        }
        
        with open(os.path.join(sample_dir, "metadata.json"), 'w') as f:
            json.dump(initial_config, f, indent=2)

In [ ]:
# Initialize the base generator with desired parameters
base_generator = SyntheticDataGenerator(
    grid_size=10,
    num_vehicles=1,
    num_pedestrians=1,
    num_obstacles=0
)

# Create the GNN dataset generator
dataset_generator = GNNDatasetGenerator(base_generator)

# Generate the dataset
dataset_generator.generate_dataset(
    num_samples=1000,
    num_steps=500,
    window_size=7
)

Found new no-collision sample: 1/500
Found new no-collision sample: 2/500
Found new no-collision sample: 3/500
Found new no-collision sample: 4/500
Found new no-collision sample: 5/500
Found new no-collision sample: 6/500
Found new no-collision sample: 7/500
Found new no-collision sample: 8/500
Found new no-collision sample: 9/500
Found new no-collision sample: 10/500
Found new no-collision sample: 11/500
Found new no-collision sample: 12/500
Found new no-collision sample: 13/500
Found new no-collision sample: 14/500
Found new collision sample: 1/500
Found new no-collision sample: 15/500
Found new no-collision sample: 16/500
Found new no-collision sample: 17/500
Found new no-collision sample: 18/500
Found new no-collision sample: 19/500
Found new no-collision sample: 20/500
Found new no-collision sample: 21/500
Found new no-collision sample: 22/500
Found new no-collision sample: 23/500
Found new no-collision sample: 24/500
Found new no-collision sample: 25/500
Found new no-collision sa

Found new no-collision sample: 404/500
Found new no-collision sample: 405/500
Found new no-collision sample: 406/500
Found new no-collision sample: 407/500
Found new no-collision sample: 408/500
Found new no-collision sample: 409/500
Found new no-collision sample: 410/500
Found new no-collision sample: 411/500
Found new no-collision sample: 412/500
Found new no-collision sample: 413/500
Found new no-collision sample: 414/500
Found new no-collision sample: 415/500
Found new no-collision sample: 416/500
Found new no-collision sample: 417/500
Found new no-collision sample: 418/500
Found new no-collision sample: 419/500
Found new no-collision sample: 420/500
Found new no-collision sample: 421/500
Found new no-collision sample: 422/500
Found new no-collision sample: 423/500
Found new no-collision sample: 424/500
Found new no-collision sample: 425/500
Found new no-collision sample: 426/500
Found new no-collision sample: 427/500
Found new no-collision sample: 428/500
Found new no-collision sa

Found new collision sample: 113/500
Found new collision sample: 114/500
Found new collision sample: 115/500
Found new collision sample: 116/500
Found new collision sample: 117/500
Found new collision sample: 118/500
Found new collision sample: 119/500
Found new collision sample: 120/500
Found new collision sample: 121/500
Found new collision sample: 122/500
Found new collision sample: 123/500
Found new collision sample: 124/500
Found new collision sample: 125/500
Found new collision sample: 126/500
Found new collision sample: 127/500
Found new collision sample: 128/500
Attempt 1200: Found 128 collision and 500 no-collision samples
Found new collision sample: 129/500
Found new collision sample: 130/500
Found new collision sample: 131/500
Found new collision sample: 132/500
Found new collision sample: 133/500
Found new collision sample: 134/500
Found new collision sample: 135/500
Found new collision sample: 136/500
Found new collision sample: 137/500
Found new collision sample: 138/500
F

Attempt 3800: Found 295 collision and 500 no-collision samples
Found new collision sample: 296/500
Found new collision sample: 297/500
Found new collision sample: 298/500
Attempt 3900: Found 298 collision and 500 no-collision samples
Found new collision sample: 299/500
Found new collision sample: 300/500
Found new collision sample: 301/500
Found new collision sample: 302/500
Found new collision sample: 303/500
Attempt 4000: Found 303 collision and 500 no-collision samples
Found new collision sample: 304/500
Found new collision sample: 305/500
Attempt 4100: Found 305 collision and 500 no-collision samples
Found new collision sample: 306/500
Found new collision sample: 307/500
Found new collision sample: 308/500
Found new collision sample: 309/500
Found new collision sample: 310/500
Attempt 4200: Found 310 collision and 500 no-collision samples
Found new collision sample: 311/500
Found new collision sample: 312/500
Found new collision sample: 313/500
Attempt 4300: Found 313 collision and

Found new collision sample: 430/500
Found new collision sample: 431/500
Attempt 9200: Found 431 collision and 500 no-collision samples
Found new collision sample: 432/500
Attempt 9300: Found 432 collision and 500 no-collision samples
Attempt 9400: Found 432 collision and 500 no-collision samples
Attempt 9500: Found 432 collision and 500 no-collision samples
Attempt 9600: Found 432 collision and 500 no-collision samples
Found new collision sample: 433/500
Attempt 9700: Found 433 collision and 500 no-collision samples
Attempt 9800: Found 433 collision and 500 no-collision samples
Found new collision sample: 434/500
Found new collision sample: 435/500
Attempt 9900: Found 435 collision and 500 no-collision samples
Found new collision sample: 436/500
Attempt 10000: Found 436 collision and 500 no-collision samples
Found new collision sample: 437/500
Attempt 10100: Found 437 collision and 500 no-collision samples
Found new collision sample: 438/500
Found new collision sample: 439/500
Found ne

Attempt 18500: Found 495 collision and 500 no-collision samples
Found new collision sample: 496/500
Found new collision sample: 497/500
Attempt 18600: Found 497 collision and 500 no-collision samples
Found new collision sample: 498/500
Found new collision sample: 499/500
Attempt 18700: Found 499 collision and 500 no-collision samples
Attempt 18800: Found 499 collision and 500 no-collision samples
Attempt 18900: Found 499 collision and 500 no-collision samples
Found new collision sample: 500/500
Saving 500 collision samples...
Saving 500 no-collision samples...
